In [2]:
import os
import sys
import numpy as np

sys.path.append(os.path.join(os.getcwd(), '../'))
from utilities.sqlcontroller import SqlController

animal = 'MD589'
thumbnail = False
downscale_factor = [1, 32][thumbnail]

CSV_DIR_PATH = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/foundation_brain_annotations'
IMAGE_DIR_PATH = f'/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/{animal}/preps/CH1/{["full", "thumbnail"][thumbnail]}'

sqlController = SqlController(animal
resolution = sqlController.scan_run.resolution * downscale_factor
scales = np.array([resolution, resolution, 20]) * 1000
aligned_shape = (np.array((sqlController.scan_run.width, sqlController.scan_run.height)) / downscale_factor).astype(int)
num_section = len(os.listdir(IMAGE_DIR_PATH))

DIR = os.path.join(os.path.expanduser('~'))
vertices_path = os.path.join(DIR, f'{animal}_aligned_section_structure_vertices_{["full", "thumbnail"][thumbnail]}.pickle')
precompute_path = os.path.join(DIR, f'{animal}_annotations_{["full", "thumbnail"][thumbnail]}')
layer_type = 'segmentation'
offset = (0, 0, 0)

known_foundation_structures = ['MVePC', 'DTgP', 'VTA', 'Li', 'Op', 'Sp5C', 'RPC', 'MVeMC', 'APT', 'IPR', 
                               'Cb', 'pc', 'Amb', 'SolIM', 'Pr5VL', 'IPC', '8n', 'MPB', 'Pr5', 'SNR', 
                               'DRD', 'PBG', '10N', 'VTg', 'R', 'IF', 'RR', 'LDTg', '5TT', 'Bar', 
                               'Tz', 'IO', 'Cu', 'SuVe', '12N', '6N', 'PTg', 'Sp5I', 'SNC', 'MnR', 
                               'RtTg', 'Gr', 'ECu', 'DTgC', '4N', 'IPA', '3N', '7N', 'LC', '7n', 
                               'SC', 'LPB', 'EW', 'Pr5DM', 'VCA', '5N', 'Dk', 'DTg', 'LVe', 'SpVe', 
                               'MVe', 'LSO', 'InC', 'IC', 'Sp5O', 'DC', 'Pn', 'LRt', 'RMC', 'PF', 
                               'VCP', 'CnF', 'Sol', 'IPL', 'X', 'AP', 'MiTg', 'DRI', 'RPF', 'VLL']
db_structures = sqlController.get_structures_dict()
non_db_structures = [structure for structure in known_foundation_structures if structure not in db_structures.keys()]

def get_structure_number(structure):
    if structure in db_structures:
        color = db_structures[structure][1]
    elif structure in non_db_structures:
        color = len(db_structures) + non_db_structures.index(structure) + 1
    else:
        color = 255
    return color

segmentation_properties = [(number, f'{structure}: {label}') for structure, (label, number, sided) in db_structures.items()]
segmentation_properties += [(len(db_structures) + index + 1, structure) for index, structure in enumerate(non_db_structures)]

Connecting dklab@db.dk.ucsd.edu:3306


In [3]:
import pickle 
import cv2
from tqdm import tqdm 

with open(vertices_path, 'rb') as file:
    aligned_section_structure_vertices = pickle.load(file)
        
def draw_numpy(section_start, section_end):
    volume = np.zeros((aligned_shape[1], aligned_shape[0], section_end - section_start), dtype=np.uint8)
    for section in tqdm(range(section_start, section_end)):
        if section in aligned_section_structure_vertices:
            template = np.zeros((aligned_shape[1], aligned_shape[0]), dtype=np.uint8)
            for structure in aligned_section_structure_vertices[section]:
                points = aligned_section_structure_vertices[section][structure]

                color = get_structure_number(structure)
                cv2.polylines(template, [points.astype(np.int32)], True, color, 10)

            volume[:, :, section - section_start - 1] = template
        
    volume = np.swapaxes(volume, 0, 1)
    return volume

# section_range = [0, num_section]
section_range = [200, 210]
volume = draw_numpy(section_range[0], section_range[1])

100%|██████████| 10/10 [00:19<00:00,  1.91s/it]


In [4]:
from VolumeNeuroglancer import VolumeNeuroglancer

NG_DIR_PATH = os.path.join('/net/birdstore/Active_Atlas_Data/data_root/pipeline_data/', animal, 'neuroglancer_data')
# NG_DIR_PATH = os.path.expanduser('~') 
precompute_path = os.path.join(NG_DIR_PATH, f'{animal}_{["full", "thumbnail"][thumbnail]}_annotations')
layer_type = 'segmentation'

def get_ranges(start, end, interval):
    ranges = []
    
    cur = start
    while cur + interval <= end:
        next = cur + interval
        ranges.append((cur, next))
        cur = next
    ranges.append((cur, end))
    return ranges

volume_ng = VolumeNeuroglancer(volume, scales, layer_type, parallel=False)
volume_ng.init_precomputed(precompute_path, num_section=num_section)

In [ ]:
for start, end in tqdm(get_ranges(0, num_section, 64)):
    vol = draw_numpy(start, end)
    volume_ng.append_volume(vol, start)

  0%|          | 0/64 [00:00<?, ?it/s]